In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.cm as mcm
import networkx as nx
from pylab import rcParams
from math import *
import time
from numpy import linalg as LA
from sklearn.datasets import fetch_openml
from math import log, exp, ceil, sqrt
from sklearn.model_selection import train_test_split
import warnings
import json
warnings.filterwarnings('ignore', category=UserWarning)

UNDIRECTED STOCHASTIC CASE (alg 2):
- to generate data, treat b_i as a random variable (see paper), sample from dataset U(1,10) (uniform 1-10)
- also change plots: need expectation
- expectation of b_i = 5.5, for plots can use a deterministic function with 5.5 replaced everywhere

- run each for 10 samplepaths (tweak as needed for good plots) --- plot with confidence intervals
p 2 different settings for graph: petersen (10 nodes) and random (100 nodes)

In [2]:
# UNDIRECTED STOCHASTIC ALGORITHM
# --------------------------------

In [3]:
# Setting up data
# ---------------

# 5 x 11
def calculate_mean_and_confidence_interval(data, t=4.596):
    mean = np.mean(data, axis=0) # calculate mean along rows of data
    stdev = np.std(data, axis=0)
    confidence_interval = t * stdev / sqrt(data.shape[0])
    return mean, confidence_interval

np.random.seed(1)
reg_weight = 0.1
moreau = 0.1
N = 100

"""
returns:
a_values (vector size N)
b_values (vector size N)
C matrix (matrix size NxN)
upper box constraints (vector size N)
lower box constraints (vector size N)
"""
# a_i ~U[1,10] for i = 0..14, and a_i = 0 for i = 15..19
# more generally, a_i = 0 for i = (n-5)..(n-1)
a_values = np.random.uniform(low=1, high=10, size=N)
a_values[-5:] = 0

# b_i is constantly sampled ~U[1,10] for all i = 0..(n-1)
def sample_b():
    return np.random.uniform(low=1, high=100, size=N)

# C is a positive definite matrix
A = np.random.randn(N,N)
B = A.dot(A.T)                                           # B is positive semidefinite
eigvals = LA.eigvals(B)
C = B + np.diagflat([min(eigvals)+1 for _ in range(N)]) # C is positive definite

# lower box constraint cannot exceed the upper box constraint
upper_box_constraints = np.random.uniform(low=50, high=100, size=N)
lower_box_constraints = np.random.uniform(low=0, high=upper_box_constraints, size=N)

In [4]:
# Graph generation
# ----------------

def petersen_graph():
    G = nx.petersen_graph()
    beta = 0.6
    W = np.diagflat(np.ones(10) * beta)
    for i in G:
        for j in G.neighbors(i):
            W[i,j] = (1-beta)/3
    return G, W

def max_degree_weights(L: np.ndarray, **kwargs):
    m, m = L.shape
    alpha = 0.5 / np.max(L)
    W = np.zeros((m,m))
    for i in range(m):
        for j in range(m):
            if i == j:
                d_i = L[i,j]
                W[i,j] = 1 - (d_i * alpha)
            elif L[i,j] != 0:
                W[i,j] = alpha
    return W

def random_graph(v, num_e):
    e = np.clip(num_e, v-1, v*(v-1)//2)
    seed = 42
    rng = np.random.default_rng(seed)
    G = nx.random_tree(v, seed=seed)
    i = v-1
    while i < e:
        v_a, v_b = tuple(rng.integers(v, size=2))
        if not G.has_edge(v_a, v_b) and v_a != v_b:
            G.add_edge(v_a, v_b)
            i += 1
    L = nx.laplacian_matrix(G).todense()
    print(L)
    W = max_degree_weights(L)
    print(W)
    return G, W

In [5]:
# Mappings and Gradient definitions
# ---------------------------------

"""
input: x is a vector size N
output: returns vector size N
"""
def grad_f_i(x, i, b_values):
    x_i = x[i]
    grad_f_i = C[i]*x_i
    grad_f_i[i] = a_values[i]*x_i + b_values[i] \
                    + (np.dot(C[i], x) - C[i,i]*x_i) \
                    + (x_i - np.clip(x_i, lower_box_constraints[i], upper_box_constraints[i]))/moreau
    return grad_f_i + reg_weight*x

"""
input: X_mat is a matrix NxN, X_mat[i] is the ith agent (player) vector
output: returns matrix A size N.N, where A[i] = ∇f_i(x_i) (bolded gradient f)
"""
def grad_f(X_mat, b_values):
    grad_f = np.zeros((N,N))
    for i in range(N):
        grad_f[i] = grad_f_i(X_mat[i], i, b_values)
    return grad_f

"""
input: x is a vector size N
output: returns sparse vector size N, where only the ith element is nonzero
"""
def F_i(x, i, b_values):
    F_i = np.zeros(N)
    x_i = x[i]
    F_i[i] = a_values[i]*x_i + b_values[i] \
               + (np.dot(C[i], x) - C[i,i]*x_i) \
               + (x_i - np.clip(x_i, lower_box_constraints[i], upper_box_constraints[i]))/moreau
    return F_i

"""
input: X_mat is a matrix NxN, X_mat[i] is the ith agent (player) vector
output: returns matrix size NxN: this is the bold F in the algorithm, not the unbolded F in the problem formulation
"""
def F_fun(X_mat, b_values):
    F = np.zeros((N,N))
    for i in range(N):
        F[i] = F_i(X_mat[i], i, b_values)
    return F

In [6]:
# Deterministic (expected) mapping and gradient definitions --- for plotting/comparison purposes only
# ---------------------------------

expected_b_value = 500.5

def expected_f_fun(x):
    out = 0
    for i in range(len(x)):
        out += expected_f_i(x,i)
    return out

def expected_f_i(x, i):
    x_i = x[i]
    return 0.5*a_values[i]*x_i**2 + expected_b_value*x_i \
               + (np.dot(C[i], x) - C[i,i]*x_i)*x_i \
               + LA.norm(x_i - np.clip(x_i, lower_box_constraints[i], upper_box_constraints[i]))**2 / (2*moreau) \
               + 0.5*reg_weight*LA.norm(x)**2

def expected_F_i(x, i):
    F_i = np.zeros(N)
    x_i = x[i]
    F_i[i] = a_values[i]*x_i + expected_b_value \
               + (np.dot(C[i], x) - C[i,i]*x_i) \
               + (x_i - np.clip(x_i, lower_box_constraints[i], upper_box_constraints[i]))/moreau
    return F_i

def expected_F_fun(X_mat):
    F = np.zeros((N,N))
    for i in range(N):
        F[i] = expected_F_i(X_mat[i], i)
    return F

def expected_F_sum(x_avg_vec):
    F_vec = np.zeros(N)
    for i in range(N):
        F_vec += expected_F_i(x_avg_vec, i)
    return F_vec

In [7]:
def DSGT(
    max_iter: int,
    epoch_size: int,
    X_0: np.ndarray,
    W: np.ndarray,
    a_update_param,
    b_update_param,
    Gamma):

    m,_ = X_0.shape
    lower_vals = np.zeros(epoch_size + 1)
    delta_x_vals = np.zeros(epoch_size + 1)
    upper_vals = np.zeros(epoch_size + 1)
    consensus_vals = np.zeros(epoch_size + 1)
    X_next = X_0
    
    # update rules for stepsize (gamma_k) and regularization parameter (lambda_k)
    stepsize_rule = lambda x : 1e-4 / ((x+Gamma)**a_update_param)      # gamma_k
    regularizer_rule = lambda x : 1e-3 / ((x+Gamma)**b_update_param)    # lambda_k

    # sample the first set of b_values and perform mapping/gradient calculations
    b_values = sample_b()
    F_mat_next = F_fun(X_next, b_values)
    grad_f_mat_next = grad_f(X_next, b_values)
    
    # Y_0 is initialized to the gradient of the base function
    Y_next = F_mat_next + regularizer_rule(0) * grad_f(X_0, b_values)
    epoch_idx = 0
    
    for k in range(1, max_iter + 2):
        # updates:
        stepsize = stepsize_rule(k) * np.eye(m)
        lambda_k_now, lambda_k_next = regularizer_rule(k), regularizer_rule(k+1)
        X_now, Y_now = X_next, Y_next
        F_mat_now, grad_f_mat_now = F_mat_next, grad_f_mat_next

        # update X:
        X_next = W.dot(X_now - np.dot(stepsize, Y_now))

        # take new sample:
        b_values = sample_b()
        
        # update gradients:
        F_mat_next = F_fun(X_next, b_values)
        grad_f_mat_next = grad_f(X_next, b_values)
        
        # update Y tracker:
        Y_next = W.dot(Y_now) + (F_mat_next - F_mat_now) + (lambda_k_next * grad_f_mat_next - lambda_k_now * grad_f_mat_now)
        
        if max_iter == 0 or ((k-1) % ceil(max_iter/epoch_size)) == 0:
            x_ave_now = np.sum(X_now, axis=0) / m
            x_ave_next = np.sum(X_next, axis=0) / m

            print(f"epoch: {k}")

            lower_vals[epoch_idx] = LA.norm(expected_F_sum(x_ave_now))
            delta_x_vals[epoch_idx] = LA.norm(x_ave_next-x_ave_now)
            upper_vals[epoch_idx] = expected_f_fun(x_ave_now)
            consensus_vals[epoch_idx] = LA.norm(X_now - np.dot(np.ones((m,1)), x_ave_now.reshape(1,m)))
            epoch_idx += 1
            
    return lower_vals, delta_x_vals, upper_vals, consensus_vals

In [8]:
a_b_params = [(0.5, 0.3), (0.6, 0.25), (0.675, 0.2)]
runcolor  = { (0.5, 0.3): 'r', (0.6, 0.25): 'g', (0.675, 0.2): 'b' }
runmarker = { (0.5, 0.3): 'o', (0.6, 0.25): 'v', (0.675, 0.2): 's' }

Gamma = 10

max_iter = 10_000
epoch_size = 10
num_samplepaths = 5
X_0 = np.random.rand(N,N) * 100

if N == 10:
    G, W = petersen_graph()
elif N == 100:
    G, W = random_graph(N, N * log(N))

lower_vals_results = {}
delta_x_vals_results = {}
upper_vals_results = {}
consensus_vals_results = {}

for (a,b) in a_b_params:
    lower_vals, delta_x_vals, upper_vals, consensus_vals = [np.zeros((num_samplepaths, epoch_size+1)) for _ in range(4)]
    for i in range(num_samplepaths):
        path_lv, path_dxv, path_uv, path_cv = DSGT(max_iter, epoch_size, X_0, W, a, b, Gamma)
        lower_vals[i] = path_lv
        delta_x_vals[i] = path_dxv
        upper_vals[i] = path_uv
        consensus_vals[i] = path_cv
    lower_vals_results[(a,b)] = calculate_mean_and_confidence_interval(lower_vals)
    delta_x_vals_results[(a,b)] = calculate_mean_and_confidence_interval(delta_x_vals)
    upper_vals_results[(a,b)] = calculate_mean_and_confidence_interval(upper_vals)
    consensus_vals_results[(a,b)] = calculate_mean_and_confidence_interval(consensus_vals)

[[ 8  0  0 ... -1  0  0]
 [ 0  5  0 ...  0  0  0]
 [ 0  0  8 ...  0  0  0]
 ...
 [-1  0  0 ...  8  0  0]
 [ 0  0  0 ...  0  5  0]
 [ 0  0  0 ...  0  0  7]]
[[0.76470588 0.         0.         ... 0.02941176 0.         0.        ]
 [0.         0.85294118 0.         ... 0.         0.         0.        ]
 [0.         0.         0.76470588 ... 0.         0.         0.        ]
 ...
 [0.02941176 0.         0.         ... 0.76470588 0.         0.        ]
 [0.         0.         0.         ... 0.         0.85294118 0.        ]
 [0.         0.         0.         ... 0.         0.         0.79411765]]
epoch: 1
epoch: 1001
epoch: 2001
epoch: 3001
epoch: 4001
epoch: 5001
epoch: 6001
epoch: 7001
epoch: 8001
epoch: 9001
epoch: 10001
epoch: 1
epoch: 1001
epoch: 2001
epoch: 3001
epoch: 4001
epoch: 5001
epoch: 6001
epoch: 7001
epoch: 8001
epoch: 9001
epoch: 10001
epoch: 1
epoch: 1001
epoch: 2001
epoch: 3001
epoch: 4001
epoch: 5001
epoch: 6001
epoch: 7001
epoch: 8001
epoch: 9001
epoch: 10001
epoch: 1
e

In [9]:
# CHANGE THIS FOR EACH PLOT STYLE
path = "Alg2_Petersen_Graph" if N == 10 else "Alg2_Random_Graph"
graph_prefix = "petersen" if N == 10 else "random"

save_plots = True

In [ ]:
# LOWER LEVEL OBJECTIVE PLOTS

x_ax = list(range(0,max_iter+1,ceil(max_iter/epoch_size)))
fig0, ax0 = plt.subplots()
fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()

ax0.sharey(ax1)
ax1.sharey(ax2)
ax1.autoscale()
ax2.autoscale()

print('(a,b) == ')

key = a_b_params[0]
print(key)
ax0.plot(x_ax, lower_vals_results[key][0], lw=2.5, marker=runmarker[key], markersize=8, color=runcolor[key], label=f"(a,b)={key}")
ax0.fill_between(x_ax, lower_vals_results[key][0] - lower_vals_results[key][1], lower_vals_results[key][0] + lower_vals_results[key][1], color=runcolor[key], alpha=0.1)
ax0.grid(True)
ax0.set_xlabel('Iteration', color='#1C2833',fontsize=15)
if save_plots:
    fig0.savefig(f"./{path}/{graph_prefix}_lower_level_{key[0]}_{key[1]}.pdf", dpi=600, format="pdf")

key = a_b_params[1]
print(key)
ax1.plot(x_ax, lower_vals_results[key][0], lw=2.5, marker=runmarker[key], markersize=8, color=runcolor[key], label=f"(a,b)={key}")
ax1.fill_between(x_ax, lower_vals_results[key][0] - lower_vals_results[key][1], lower_vals_results[key][0] + lower_vals_results[key][1], color=runcolor[key], alpha=0.1)
ax1.grid(True)
ax1.set_xlabel('Iteration', color='#1C2833',fontsize=15)
if save_plots:
    fig1.savefig(f"./{path}/{graph_prefix}_lower_level_{key[0]}_{key[1]}.pdf", dpi=600, format="pdf")

key = a_b_params[2]
print(key)
ax2.plot(x_ax, lower_vals_results[key][0], lw=2.5, marker=runmarker[key], markersize=8, color=runcolor[key], label=f"(a,b)={key}")
ax2.fill_between(x_ax, lower_vals_results[key][0] - lower_vals_results[key][1], lower_vals_results[key][0] + lower_vals_results[key][1], color=runcolor[key], alpha=0.1)
ax2.grid(True)
ax2.set_xlabel('Iteration', color='#1C2833',fontsize=15)
if save_plots:
    fig2.savefig(f"./{path}/{graph_prefix}_lower_level_{key[0]}_{key[1]}.pdf", dpi=600, format="pdf")

#plt.ylabel(r"$\|F(\mathbf{\bar{x}})\|_2$", color='#1C2833',fontsize=18)
#plt.title("lower level objective", fontsize=18)
#plt.legend()

plt.show()

In [ ]:
# CHANGE IN STEP PLOTS

fig0, ax0 = plt.subplots()
fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()
lims = (-9.45834334086389, -3.834118554164067)

print('(a,b) == ')

key = a_b_params[0]
print(key)

ax0.plot(x_ax, np.log(delta_x_vals_results[key][0]), lw=2.5, marker=runmarker[key], markersize=8, color=runcolor[key], label=f"(a,b)={key}")
ax0.fill_between(x_ax, np.log(delta_x_vals_results[key][0] - delta_x_vals_results[key][1]), np.log(delta_x_vals_results[key][0] + delta_x_vals_results[key][1]), color=runcolor[key], alpha=0.1)
ax0.grid(True)
ax0.set_xlabel('Iteration', color='#1C2833',fontsize=15)
print(ax0.get_ylim())
ax0.set_ylim(lims)
if save_plots:
    fig0.savefig(f"./{path}/{graph_prefix}_delta_step_{key[0]}_{key[1]}.pdf", dpi=600, format="pdf")

key = a_b_params[1]
print(key)
ax1.plot(x_ax, np.log(delta_x_vals_results[key][0]), lw=2.5, marker=runmarker[key], markersize=8, color=runcolor[key], label=f"(a,b)={key}")
ax1.fill_between(x_ax, np.log(delta_x_vals_results[key][0] - delta_x_vals_results[key][1]), np.log(delta_x_vals_results[key][0] + delta_x_vals_results[key][1]), color=runcolor[key], alpha=0.1)
ax1.grid(True)
ax1.set_xlabel('Iteration', color='#1C2833',fontsize=15)
ax1.set_ylim(lims)
if save_plots:
    fig1.savefig(f"./{path}/{graph_prefix}_delta_step_{key[0]}_{key[1]}.pdf", dpi=600, format="pdf")

key = a_b_params[2]
print(key)
ax2.plot(x_ax, np.log(delta_x_vals_results[key][0]), lw=2.5, marker=runmarker[key], markersize=8, color=runcolor[key], label=f"(a,b)={key}")
ax2.fill_between(x_ax, np.log(delta_x_vals_results[key][0] - delta_x_vals_results[key][1]), np.log(delta_x_vals_results[key][0] + delta_x_vals_results[key][1]), color=runcolor[key], alpha=0.1)
ax2.grid(True)
ax2.set_xlabel('Iteration', color='#1C2833',fontsize=15)
ax2.set_ylim(lims)
print(ax2.get_ylim())
if save_plots:
    fig2.savefig(f"./{path}/{graph_prefix}_delta_step_{key[0]}_{key[1]}.pdf", dpi=600, format="pdf")

plt.show()

In [ ]:
# UPPER LEVEL PLOTS

fig0, ax0 = plt.subplots()
fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()

ax0.sharey(ax1)
ax1.sharey(ax2)
ax1.autoscale()
ax2.autoscale()

print('(a,b) == ')

key = a_b_params[0]
print(key)
ax0.plot(x_ax, upper_vals_results[key][0], lw=2.5, marker=runmarker[key], markersize=8, color=runcolor[key], label=f"(a,b)={key}")
ax0.fill_between(x_ax, upper_vals_results[key][0] - upper_vals_results[key][1], upper_vals_results[key][0] + upper_vals_results[key][1], color=runcolor[key], alpha=0.1)
ax0.grid(True)
ax0.set_xlabel('Iteration', color='#1C2833',fontsize=15)
if save_plots:
    fig0.savefig(f"./{path}/{graph_prefix}_upper_level_{key[0]}_{key[1]}.pdf", dpi=600, format="pdf")

key = a_b_params[1]
print(key)
ax1.plot(x_ax, upper_vals_results[key][0], lw=2.5, marker=runmarker[key], markersize=8, color=runcolor[key], label=f"(a,b)={key}")
ax1.fill_between(x_ax, upper_vals_results[key][0] - upper_vals_results[key][1], upper_vals_results[key][0] + upper_vals_results[key][1], color=runcolor[key], alpha=0.1)
ax1.grid(True)
ax1.set_xlabel('Iteration', color='#1C2833',fontsize=15)
if save_plots:
    fig1.savefig(f"./{path}/{graph_prefix}_upper_level_{key[0]}_{key[1]}.pdf", dpi=600, format="pdf")

key = a_b_params[2]
print(key)
ax2.plot(x_ax, upper_vals_results[key][0], lw=2.5, marker=runmarker[key], markersize=8, color=runcolor[key], label=f"(a,b)={key}")
ax2.fill_between(x_ax, upper_vals_results[key][0] - upper_vals_results[key][1], upper_vals_results[key][0] + upper_vals_results[key][1], color=runcolor[key], alpha=0.1)
ax2.grid(True)
ax2.set_xlabel('Iteration', color='#1C2833',fontsize=15)
if save_plots:
    fig2.savefig(f"./{path}/{graph_prefix}_upper_level_{key[0]}_{key[1]}.pdf", dpi=600, format="pdf")

plt.show()

In [ ]:
# CONSENSUS ERROR PLOTS

fig0, ax0 = plt.subplots()
fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()
limits = (-10.7089277594598, 8.850778308287357)

print('(a,b) == ')

key = a_b_params[0]
print(key)
ax0.plot(x_ax, np.log(consensus_vals_results[key][0]), lw=2.5, marker=runmarker[key], markersize=8, color=runcolor[key], label=f"(a,b)={key}")
ax0.fill_between(x_ax, np.log(consensus_vals_results[key][0] - consensus_vals_results[key][1]), np.log(consensus_vals_results[key][0] + consensus_vals_results[key][1]), color=runcolor[key], alpha=0.1)
ax0.grid(True)
ax0.set_xlabel('Iteration', color='#1C2833',fontsize=15)
ax0.set_ylim(limits)
if save_plots:
    fig0.savefig(f"./{path}/{graph_prefix}_consensus_err_{key[0]}_{key[1]}.pdf", dpi=600, format="pdf")

key = a_b_params[1]
print(key)
ax1.plot(x_ax, np.log(consensus_vals_results[key][0]), lw=2.5, marker=runmarker[key], markersize=8, color=runcolor[key], label=f"(a,b)={key}")
ax1.fill_between(x_ax, np.log(consensus_vals_results[key][0] - consensus_vals_results[key][1]), np.log(consensus_vals_results[key][0] + consensus_vals_results[key][1]), color=runcolor[key], alpha=0.1)
ax1.grid(True)
ax1.set_xlabel('Iteration', color='#1C2833',fontsize=15)
ax1.set_ylim(limits)
if save_plots:
    fig1.savefig(f"./{path}/{graph_prefix}_consensus_err_{key[0]}_{key[1]}.pdf", dpi=600, format="pdf")

key = a_b_params[2]
print(key)
ax2.plot(x_ax, np.log(consensus_vals_results[key][0]), lw=2.5, marker=runmarker[key], markersize=8, color=runcolor[key], label=f"(a,b)={key}")
ax2.fill_between(x_ax, np.log(consensus_vals_results[key][0] - consensus_vals_results[key][1]), np.log(consensus_vals_results[key][0] + consensus_vals_results[key][1]), color=runcolor[key], alpha=0.1)
ax2.grid(True)
ax2.set_xlabel('Iteration', color='#1C2833',fontsize=15)
ax2.set_ylim(limits)
if save_plots:
    fig2.savefig(f"./{path}/{graph_prefix}_consensus_err_{key[0]}_{key[1]}.pdf", dpi=600, format="pdf")

plt.show()

In [ ]:
descriptor = "petersen" if N == 10 else "random"
fig = plt.figure(figsize=(10, 10))

if N == 10:
    nx.draw_networkx(G, node_size=200, with_labels=False)
else:
    pos = nx.kamada_kawai_layout(G)
    nx.draw_networkx(G, node_size=100, with_labels=False, pos=pos)
plt.title(r"induced graph $\mathcal{G}_\mathbf{R}$ ")
plt.show()

if save_plots:
    fig.savefig(f"./{path}/R_{descriptor}_graph_{N}_nodes.pdf", dpi=600, format="pdf")